**Análise Exploratória dos Dados (Data Understanding)**

In [ ]:
import pandas as pd
df = pd.read_csv('/content/streaming_data.csv')
print(df.head())


In [ ]:
print(df.describe())

In [ ]:
print(df.info())

In [ ]:
print(df.isna().sum())

**Tratamento dos Dados (Data Preparation)**

In [ ]:
# Substituir NaN por 0
df['Subscription_type'].fillna(0, inplace=True)
df['Num_streaming_services'].fillna(0, inplace=True)
df['Churned'].fillna(0, inplace=True)
df['Avg_rating'].fillna(0, inplace=True)
df['Devices_connected'].fillna(0, inplace=True)

In [ ]:
# Remover linhas nulas
df.dropna(subset=['Gender', 'Subscription_type', 'Age'], inplace=True)

In [ ]:
# Conversão de Churned
df['Churned'] = df['Churned'].replace({0: 'No', 1: 'Yes'})

In [ ]:
# Crie um dicionário de mapeamento para os tipos de assinatura
subscription_mapping = {'Basic': 1, 'Standard': 2, 'Premium': 3}

# Substitua os valores da coluna 'Subscription_type' usando o mapeamento
df['Subscription_type'] = df['Subscription_type'].map(subscription_mapping)

# Preencha os valores NaN na coluna 'Subscription_type' com 0
df['Subscription_type'].fillna(0, inplace=True)

# Converta a coluna 'Subscription_type' para inteiro
df = df.astype({'Subscription_type': 'int', 'Num_streaming_services': 'int'})

**Modelagem - Regressão Logística**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay


In [ ]:
# Definir X e y
X = df.drop(columns=['User_id', 'Churned'])
y = df['Churned']


In [ ]:
# Codificar variáveis categóricas
X = pd.get_dummies(X, drop_first=True)
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Escalonamento
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Verifique se há valores infinitos em X_train
!pip install numpy
import numpy as np

# Verifique se há inf e -inf e substitua por NaN
X_train = np.nan_to_num(X_train, nan=np.nan, posinf=np.nan, neginf=np.nan)

# Impute NaNs usando SimpleImputer
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean') # Você pode escolher uma estratégia diferente
X_train = imputer.fit_transform(X_train)

In [ ]:
# Use HistGradientBoostingClassifier
!pip install scikit-learn
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier()
model.fit(X_train, y_train)

In [ ]:
# Regressão Logística
model = LogisticRegression()
model.fit(X_train, y_train)


In [ ]:
# Verifique se há valores infinitos em X_test
# Verifique se há inf e -inf e substitua por NaN
X_test = np.nan_to_num(X_test, nan=np.nan, posinf=np.nan, neginf=np.nan)

# Impute NaNs usando SimpleImputer
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean') # You can choose a different strategy
X_test = imputer.fit_transform(X_test)

# Regressão Logística
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)


In [ ]:
# Previsões
y_pred = model.predict(X_test)

In [ ]:
# Matriz de confusão
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

**Tunning**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay

# Definir os hiperparâmetros que queremos testar
param_grid = {
    'n_estimators': [100, 200, 300],  # número de árvores
    'max_depth': [None, 10, 20, 30 ],  # profundidade máxima das árvores
    'min_samples_split': [2, 5, 10],  # mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4]     # mínimo de amostras em uma folha
}


In [ ]:
# Inicializar o modelo RandomForest
rf_model = RandomForestClassifier(random_state=42)

In [ ]:
# Inicializar o GridSearchCV com 5 folds na validação cruzada
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy')


In [ ]:
# Converter X_train para um DataFrame pandas (se ainda não for)
X_train = pd.DataFrame(X_train)

# Remover linhas com valores NaN
X_train.dropna(inplace=True)

# Converter X_train de volta para um array numpy (se necessário)
X_train = X_train.values

In [ ]:
from sklearn.impute import SimpleImputer

# Inicializar o imputer com a estratégia desejada (média, mediana ou moda)
imputer = SimpleImputer(strategy='mean')

# Ajustar o imputer aos dados de treino e transformar X_train
X_train = imputer.fit_transform(X_train)

In [ ]:
# Ajustar (fit) o GridSearch nos dados de treino
grid_search.fit(X_train, y_train)

In [ ]:
# Obter os melhores parâmetros encontrados
best_params = grid_search.best_estimator_.get_params()
print("Melhores Parâmetros Encontrados:", best_params)


In [ ]:
# Fazer previsões com o melhor modelo
y_pred = grid_search.predict(X_test)

In [ ]:
# Plotar a matriz de confusão
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)


**Modelagem - Random Forest**

In [ ]:
# Codificar variáveis categóricas com get_dummies
X = pd.get_dummies(X, drop_first=True)  # One-hot encoding


In [ ]:
# Codificar a variável target (churned) com LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)  # Transforma 'Yes'/'No' em 1 e 0


In [ ]:
# Dividir o dataset em treino e teste (70% treino, 30% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Normalização das variáveis com MinMaxScaler
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Criar e treinar o modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
# Fazer previsões nos dados de teste
y_pred = rf_model.predict(X_test)

In [ ]:
# Plotar a matriz de confusão
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
